# Voyages API Example

## Run this example in [Colab](https://colab.research.google.com/github/SignalOceanSdk/SignalSDK/blob/master/docs/examples/jupyter/VoyagesAPI/VoyagesAPI.ipynb). 

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [22]:
!pip install signal-ocean

In [23]:
signal_ocean_api_key = '' #replace with your subscription key

## Call the Voyages API
The Voyages API retrieves information about vessel voyages.

In [2]:
from signal_ocean import Connection
from signal_ocean.voyages import VoyagesAPI
from signal_ocean.voyages import Vessel, VesselFilter
from signal_ocean.voyages import VesselType, VesselTypeFilter
from signal_ocean.voyages import VesselClass, VesselClassFilter
import pandas as pd
from datetime import date, timedelta

In [4]:
connection = Connection(signal_ocean_api_key)
api = VoyagesAPI(connection)

### Get voyages for vessel

In [7]:
# get the imo by using the vessel name
vessel = api.get_imos(VesselFilter('Seaking'))[0]
vessel

Vessel(imo=9292187, vessel_name='Seaking')

In [8]:
vessel_voyages = api.get_voyages(imo=vessel.imo)

In [9]:
vessel_voyages_df = pd.DataFrame([v.__dict__ for v in vessel_voyages])
vessel_voyages_df.tail(5)

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,horizon,latest_received_ais,vessel_name,vessel_type,vessel_class,trade,trade_id,vessel_status,commercial_operator,start_date,first_load_arrival_date,end_date,charterer_id,charterer,rate,rate_type,ballast_bonus,ballast_bonus_type,cargo_type_id,cargo_type,cargo_group_id,cargo_group,cargo_type_source,quantity,laycan_from,laycan_to,fixture_status_id,fixture_status,fixture_date,fixture_is_coa,fixture_is_hold,is_implied_by_ais,has_manual_entries,ballast_distance,predicted_ballast_distance,laden_distance,predicted_laden_distance
53,9292187,54,1,84,1,1684,False,"(VoyageEvent(id='I8DC99BSED81E9100', port_id=3...",I8DC99BVED81E9100,Historic,NaT,Seaking,Tanker,VLCC,Crude,1,Voyage,Thenamaris,2021-05-03 11:59:38+00:00,2021-06-01 15:55:03+00:00,2021-07-01 11:54:33+00:00,529.0,ExxonMobil,36.25,WS,None,None,19,Crude Oil,130000.0,Dirty,MarketInfo,270000.0,2021-06-01 00:00:00+00:00,2021-06-03 23:59:59+00:00,1.0,FullyFixed,2021-05-25 08:33:41.867000+00:00,False,False,None,None,5332.30,None,3985.42,NaN
54,9292187,55,1,84,1,1684,False,"(VoyageEvent(id='I8DC99BSED86DAB00', port_id=3...",I8DC99BVED86DAB00,Historic,NaT,Seaking,Tanker,VLCC,Crude,1,Voyage,Thenamaris,2021-07-01 11:54:33+00:00,2021-07-19 15:59:45+00:00,2021-08-11 15:57:40+00:00,1338.0,PTT Public Company Limited,29.50,WS,None,None,19,Crude Oil,130000.0,Dirty,MarketInfo,270000.0,2021-07-18 00:00:00+00:00,2021-07-20 23:59:59+00:00,1.0,FullyFixed,2021-07-08 10:08:07.783000+00:00,False,False,None,None,3674.77,None,4936.38,NaN
55,9292187,56,1,84,1,1684,False,"(VoyageEvent(id='I8DC99BSED8953800', port_id=7...",I8DC99BVED8953800,Historic,NaT,Seaking,Tanker,VLCC,Crude,1,Voyage,Thenamaris,2021-08-11 15:57:40+00:00,2021-09-07 15:26:55+00:00,2021-10-02 15:45:46+00:00,2340.0,Nghi Son Refinery and Petrochemical LLC,29.75,WS,None,None,19,Crude Oil,130000.0,Dirty,MarketInfo,270000.0,2021-09-07 00:00:00+00:00,2021-09-08 23:59:59+00:00,1.0,FullyFixed,2021-08-19 18:07:36.710000+00:00,False,False,None,None,4649.35,None,4859.08,NaN
56,9292187,57,1,84,1,1684,False,"(VoyageEvent(id='I8DC99BSED8E45200', port_id=1...",I8DC99BVED8E45200,Historic,NaT,Seaking,Tanker,VLCC,Crude,1,Voyage,Thenamaris,2021-10-02 15:45:46+00:00,2022-01-04 07:41:53+00:00,2022-01-28 19:55:03+00:00,529.0,ExxonMobil,42.50,WS,None,None,19,Crude Oil,130000.0,Dirty,MarketInfo,270000.0,2022-01-01 00:00:00+00:00,2022-01-03 00:00:00+00:00,1.0,FullyFixed,2021-12-20 13:46:13+00:00,False,False,None,None,9140.34,None,3653.72,NaN
57,9292187,58,1,84,1,1684,False,"(VoyageEvent(id='I8DC99BSED9828600', port_id=3...",I8DC99BVED9828600,Current,2022-03-01 14:35:55+00:00,Seaking,Tanker,VLCC,Crude,1,Voyage,Thenamaris,2022-01-28 19:55:03+00:00,2022-02-12 23:57:01+00:00,2022-03-10 23:00:00+00:00,1482.0,Shell,NaN,None,None,None,19,Crude Oil,130000.0,Dirty,MarketInfo,270000.0,2022-02-14 00:00:00+00:00,2022-02-16 00:00:00+00:00,1.0,FullyFixed,2022-02-04 13:58:23+00:00,False,False,None,None,3705.03,None,3505.09,488.85


In [ ]:
vessel_events_df = pd.DataFrame(e.__dict__ for voyage_events in vessel_voyages_df['events'] for e in voyage_events)
vessel_events_df.tail(5)

,id,port_id,voyage_id,event_type,event_horizon,purpose,arrival_date,sailing_date,latitude,longitude,...,area_idlevel0,area_name_level0,area_idlevel1,area_name_level1,area_idlevel2,area_name_level2,area_idlevel3,area_name_level3,low_ais_density,event_details
299,None,3532,None,PortCall,Historical,Discharge,2021-03-25 03:55:58+00:00,2021-03-26 23:59:41+00:00,22.5143,69.7044,...,24764,Pakistan / West Coast India,25012,India / Pakistan,25023,India / Pakistan,84,East,None,"(VoyageEventDetail(id=None, event_id=None, eve..."
300,None,3532,None,VoyageStart,Historical,Start,NaT,NaT,22.5143,69.7044,...,24764,Pakistan / West Coast India,25012,India / Pakistan,25023,India / Pakistan,84,East,None,None
301,None,3711,None,Stop,Historical,Stop,2021-03-29 19:59:51+00:00,2021-03-30 04:58:00+00:00,24.9521,58.6659,...,24777,Arabian Gulf,2,Arabian Gulf,89,Arabian Gulf,84,East,None,"(VoyageEventDetail(id=None, event_id=None, eve..."
302,None,3778,None,PortCall,Future,Load,2021-04-03 15:46:02.040000+00:00,2021-04-08 00:00:00+00:00,26.7658,50.1478,...,24777,Arabian Gulf,2,Arabian Gulf,89,Arabian Gulf,84,East,True,None
303,None,3823,None,PortCall,Future,Discharge,2021-04-26 15:41:50.200000+00:00,2021-04-28 16:22:09.956000+00:00,23.7871,120.1687,...,24729,Taiwan,17,China / Taiwan,99,Far East,84,East,True,None


In [ ]:
vessel_event_details_df = pd.DataFrame(e.__dict__ for event_details in vessel_events_df['event_details'] for e in event_details or [])
vessel_event_details_df.tail(5)

,id,event_id,event_detail_type,arrival_date,sailing_date,start_time_of_operation,end_time_of_operation,geo_asset_id,geo_asset_name,latitude,longitude,other_vessel_imo,other_vessel_name
271,None,None,Jetty,2021-02-03 07:55:41+00:00,2021-02-04 23:39:34+00:00,2021-02-03 10:57:50+00:00,2021-02-03 12:15:52+00:00,4369,FPSO Akpo,3.160565,6.831695,NaN,None
272,None,None,Stop,2021-02-06 11:35:06+00:00,2021-02-08 10:46:06+00:00,NaT,NaT,4062,Port Gentil Anchorage,-0.996812,7.547887,NaN,None
273,None,None,Jetty,2021-02-09 09:05:48+00:00,2021-02-28 23:06:13.400000+00:00,2021-02-26 15:27:11+00:00,2021-02-26 15:27:11+00:00,5352,Bonny Offshore Terminal,4.170430,7.218700,NaN,None
274,None,None,Jetty,2021-03-25 03:55:58+00:00,2021-03-26 23:59:41+00:00,2021-03-25 03:55:58+00:00,2021-03-26 23:59:41+00:00,5112,Essar Vadinar Refinery,22.514310,69.704380,NaN,None
275,None,None,Stop,2021-03-29 19:59:51+00:00,2021-03-30 04:58:00+00:00,NaT,NaT,5101,Mina Al Fahal Refinery,24.952147,58.665850,NaN,None


### Get voyages for vessel class

In [10]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('vlcc'))[0]
vlcc_id = vc.vessel_class_id
vlcc_id

84

In [ ]:
date_from = date.today() - timedelta(days=30)
recent_vlcc_voyages = api.get_voyages(vessel_class_id=vlcc_id, date_from=date_from)

In [ ]:
recent_vlcc_voyages = pd.DataFrame([v.__dict__ for v in recent_vlcc_voyages])
recent_vlcc_voyages.tail(5)

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,vessel_name,...,cargo_group,cargo_type_source,quantity,laycan_from,laycan_to,fixture_status_id,fixture_status,fixture_date,fixture_is_coa,fixture_is_hold
188,9878656,4,1,84,1,-1,False,"(VoyageEvent(id=None, port_id=3596, voyage_id=...",9878656.4,Shoho Maru,...,Dirty,Estimated,NaN,NaT,NaT,NaN,None,NaT,None,None
189,9878826,4,1,84,1,1713,False,"(VoyageEvent(id=None, port_id=3801, voyage_id=...",9878826.4,Babylon,...,Dirty,Estimated,NaN,NaT,NaT,NaN,None,NaT,None,None
190,9878826,5,1,84,1,1713,False,"(VoyageEvent(id=None, port_id=3801, voyage_id=...",9878826.5,Babylon,...,Dirty,Estimated,NaN,NaT,NaT,NaN,None,NaT,None,None
191,9878838,5,1,84,1,1713,False,"(VoyageEvent(id=None, port_id=3801, voyage_id=...",9878838.5,Silverstone St,...,Dirty,Estimated,NaN,NaT,NaT,NaN,None,NaT,None,None
192,9878838,6,1,84,1,1713,False,"(VoyageEvent(id=None, port_id=3801, voyage_id=...",9878838.6,Silverstone St,...,Dirty,Estimated,NaN,NaT,NaT,NaN,None,NaT,None,None


### Get voyages for vessel in flat format

In [11]:
# get the imo by using the vessel name
vessel = api.get_imos(VesselFilter('Seaking'))[0]
vessel

Vessel(imo=9292187, vessel_name='Seaking')

In [12]:
vessel_voyages_flat = api.get_voyages_flat(imo=vessel.imo)

In [ ]:
vessel_voyages_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.voyages)
vessel_voyages_df.tail(5)

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,vessel_name,...,cargo_group,cargo_type_source,quantity,laycan_from,laycan_to,fixture_status_id,fixture_status,fixture_date,fixture_is_coa,fixture_is_hold
48,9292187,49,1,84,1,1684,False,None,9292187.49,Seaking,...,Dirty,Estimated,NaN,2020-04-25 03:47:05+00:00,2020-04-26 03:47:05+00:00,1.0,FullyFixed,2020-03-19 03:42:23+00:00,False,False
49,9292187,50,1,84,1,1713,False,None,9292187.50,Seaking,...,Dirty,Estimated,NaN,2020-06-01 03:57:15+00:00,2020-06-02 03:57:15+00:00,1.0,FullyFixed,2020-05-26 17:49:57+00:00,True,True
50,9292187,51,1,84,1,1713,False,None,9292187.51,Seaking,...,Dirty,MarketInfo,270000.0,2020-10-16 00:00:00+00:00,2020-10-18 23:59:00+00:00,1.0,FullyFixed,2020-10-07 13:08:52+00:00,True,False
51,9292187,52,1,84,1,1713,False,None,9292187.52,Seaking,...,Dirty,MarketInfo,260000.0,2021-02-02 00:00:00+00:00,2021-02-02 00:00:00+00:00,1.0,FullyFixed,2021-01-12 13:45:27.563000+00:00,False,False
52,9292187,53,1,84,1,1713,False,None,9292187.53,Seaking,...,Dirty,MarketInfo,270000.0,2021-04-08 00:00:00+00:00,2021-04-08 00:00:00+00:00,1.0,FullyFixed,2021-03-23 12:57:56+00:00,False,False


In [ ]:
vessel_events_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.events)
vessel_events_df.tail(5)

,id,port_id,voyage_id,event_type,event_horizon,purpose,arrival_date,sailing_date,latitude,longitude,...,area_idlevel0,area_name_level0,area_idlevel1,area_name_level1,area_idlevel2,area_name_level2,area_idlevel3,area_name_level3,low_ais_density,event_details
299,9292187.52.6,3532,9292187.52,PortCall,Historical,Discharge,2021-03-25 03:55:58+00:00,2021-03-26 23:59:41+00:00,22.5143,69.7044,...,None,None,None,None,None,None,None,None,None,None
300,9292187.53.0,3532,9292187.53,VoyageStart,Historical,Start,NaT,NaT,22.5143,69.7044,...,None,None,None,None,None,None,None,None,None,None
301,9292187.53.1,3711,9292187.53,Stop,Historical,Stop,2021-03-29 19:59:51+00:00,2021-03-30 04:58:00+00:00,24.9521,58.6659,...,None,None,None,None,None,None,None,None,None,None
302,9292187.53.2,3778,9292187.53,PortCall,Future,Load,2021-04-03 15:46:02.040000+00:00,2021-04-08 00:00:00+00:00,26.7658,50.1478,...,None,None,None,None,None,None,None,None,True,None
303,9292187.53.3,3823,9292187.53,PortCall,Future,Discharge,2021-04-26 15:41:50.200000+00:00,2021-04-28 16:22:09.956000+00:00,23.7871,120.1687,...,None,None,None,None,None,None,None,None,True,None


In [ ]:
vessel_event_details_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.event_details)
vessel_event_details_df.tail(5)

,id,event_id,event_detail_type,arrival_date,sailing_date,start_time_of_operation,end_time_of_operation,geo_asset_id,geo_asset_name,latitude,longitude,other_vessel_imo,other_vessel_name
271,9292187.52.3.0,9292187.52.3,Jetty,2021-02-03 07:55:41+00:00,2021-02-04 23:39:34+00:00,2021-02-03 10:57:50+00:00,2021-02-03 12:15:52+00:00,4369,FPSO Akpo,3.160565,6.831695,NaN,None
272,9292187.52.4.0,9292187.52.4,Stop,2021-02-06 11:35:06+00:00,2021-02-08 10:46:06+00:00,NaT,NaT,4062,Port Gentil Anchorage,-0.996812,7.547887,NaN,None
273,9292187.52.5.0,9292187.52.5,Jetty,2021-02-09 09:05:48+00:00,2021-02-28 23:06:13.400000+00:00,2021-02-26 15:27:11+00:00,2021-02-26 15:27:11+00:00,5352,Bonny Offshore Terminal,4.170430,7.218700,NaN,None
274,9292187.52.6.0,9292187.52.6,Jetty,2021-03-25 03:55:58+00:00,2021-03-26 23:59:41+00:00,2021-03-25 03:55:58+00:00,2021-03-26 23:59:41+00:00,5112,Essar Vadinar Refinery,22.514310,69.704380,NaN,None
275,9292187.53.1.0,9292187.53.1,Stop,2021-03-29 19:59:51+00:00,2021-03-30 04:58:00+00:00,NaT,NaT,5101,Mina Al Fahal Refinery,24.952147,58.665850,NaN,None


In [ ]:
vessel_voyages_geos_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.geos)
vessel_voyages_geos_df.tail(5)

,id,name,port_id,port_name,country_id,country,area_idlevel0,area_name_level0,area_idlevel1,area_name_level1,area_idlevel2,area_name_level2,area_idlevel3,area_name_level3
205,6507,Terminal Island,3857,Long Beach,240,United States,24741,US West Coast,25017,West Coast North America,25026,West Coast North America,85,Pacific America
206,6163,Vopak Terminal Europoort,3689,Rotterdam,173,Netherlands,24758,Continent,25016,UK Continent,25025,Mediterranean / UK Continent,25028,West
207,5744,Euro Tank Terminal B.v.,3689,Rotterdam,173,Netherlands,24758,Continent,25016,UK Continent,25025,Mediterranean / UK Continent,25028,West
208,6361,Aruba Lightering Zone,3214,San Nicolaas,21,Aruba,24746,Caribs,9,Caribs,25019,Atlantic America,25028,West
209,5745,Maatschap Europoort Terminal,3689,Rotterdam,173,Netherlands,24758,Continent,25016,UK Continent,25025,Mediterranean / UK Continent,25028,West


### Get voyages for vessel class in the flat format

In [13]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('vlcc'))[0]
vlcc_id = vc.vessel_class_id

date_from = date.today() - timedelta(days=30)
recent_vlcc_voyages_flat = api.get_voyages_flat(vessel_class_id=vlcc_id, date_from=date_from)

In [14]:
print('Voyages:', len(recent_vlcc_voyages_flat.voyages))
print('Events:', len(recent_vlcc_voyages_flat.events))
print('EventDetails:', len(recent_vlcc_voyages_flat.event_details))
print('Geos:', len(recent_vlcc_voyages_flat.geos))

Voyages: 340
Events: 1080
EventDetails: 463
Geos: 246


### Get voyages for vessel incrementally 

Initial requests retrieves voyages mathcing the query criteria and a query token that can be used in the subsequent request.

In [15]:
# get the imo by using the vessel name
vessel = api.get_imos(VesselFilter('XIN YONG YANG'))[0]
vessel

Vessel(imo=9416642, vessel_name='Xin Yong Yang')

In [16]:
vessel_voyages, next_request_token = api.get_incremental_voyages(imo=vessel.imo)
len(vessel_voyages)

73

Provided the token retrieved for the previous request, the subsequent request retrieves only voyages that have been updated since the previous request. 

In [ ]:
incremental_voyages, next_request_token = api.get_incremental_voyages(imo=9416642, incremental_token=next_request_token)
len(incremental_voyages)

0

The retrieved information is used to replace any modify voyages from the dataset. Note that returned voyages marked as deleted are only used to filter out voyages. 

In [ ]:
updated_voyage_ids = set(v.id for v in incremental_voyages)
vessel_voyages = [v for v in vessel_voyages if v.id not in updated_voyage_ids]
new_vessel_voyages = [v for v in incremental_voyages if not v.deleted]
vessel_voyages = sorted(vessel_voyages + new_vessel_voyages, key= lambda v: v.voyage_number)
len(vessel_voyages)

68

### Get voyages for vessel class incrementally 

Voyages for vessel class are retieved and updated in the same way incrementally.

In [17]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('vlcc'))[0]
vessel_class_id = vc.vessel_class_id
vessel_class_id

84

In [18]:
date_from = date.today() - timedelta(days=30)

In [ ]:
voyages, next_request_token = api.get_incremental_voyages(vessel_class_id=vessel_class_id, date_from=date_from)
len(voyages)

193

In [ ]:
incremental_voyages, next_request_token = api.get_incremental_voyages(vessel_class_id=vessel_class_id, date_from=date_from, incremental_token=next_request_token)
len(incremental_voyages)

0

In [ ]:
updated_voyage_ids = set(v.id for v in incremental_voyages)
voyages = [v for v in voyages if v.id not in updated_voyage_ids]
new_voyages = [v for v in incremental_voyages if not v.deleted]
voyages = sorted(voyages + new_voyages, key= lambda v: (v.imo, v.voyage_number))
len(voyages)

193

### Get voyages for vessel class incrementally in the flat format

Voyages may be retrieved and updated incrementally in the flat format. 

In [19]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('vlcc'))[0]
vessel_class_id = vc.vessel_class_id
vessel_class_id

84

In [20]:
date_from = date.today() - timedelta(days=30)

In [21]:
voyages_flat, next_request_token = api.get_incremental_voyages_flat(vessel_class_id=vessel_class_id, date_from=date_from)

In [ ]:
voyages = voyages_flat.voyages
events = voyages_flat.events
event_details = voyages_flat.event_details
geos = voyages_flat.geos

In [ ]:
len(voyages), len(events), len(event_details), len(geos)

(193, 801, 345, 187)

In [ ]:
incremental_voyages_flat, next_request_token = api.get_incremental_voyages_flat(vessel_class_id=vessel_class_id, date_from=date_from, incremental_token=next_request_token)

In [ ]:
len(incremental_voyages_flat.voyages)

0

In this case the update step is applied to voyages, events, event details and geos datasets.

In [ ]:
updated_voyage_ids = set(v.id for v in incremental_voyages_flat.voyages)
voyages = [v for v in voyages if v.id not in updated_voyage_ids]
new_voyages = [v for v in incremental_voyages_flat.voyages if not v.deleted]
voyages = sorted(voyages + new_voyages, key= lambda v: (v.imo, v.voyage_number))
len(voyages)

193

In [ ]:
updated_event_ids = set(e.id for e in events if e.voyage_id in updated_voyage_ids)
events = [e for e in events if e.id not in updated_event_ids]
events = sorted(events + list(incremental_voyages_flat.events), key= lambda e: e.id)
len(events)

801

In [ ]:
event_details = [e for e in event_details if e.id not in updated_event_ids]
event_details = sorted(event_details + list(incremental_voyages_flat.event_details), key= lambda e: e.id)
len(event_details)

345

In [ ]:
updated_geo_ids = set(g.id for g in incremental_voyages_flat.geos)
geos = [g for g in geos if g.id not in updated_geo_ids] + list(incremental_voyages_flat.geos)
len(geos)

187